In [76]:
import pandas as pd
from analysis import *
from statsmodels.stats.proportion import proportions_ztest
from scipy.stats import binom_test

In [77]:
surge_annotations = data.surge_evaluation.annotation_dataframe()
surge_annotations_comparative = data.surge_evaluation.comparative_annotation_dataframe()

surge_annotations

0  \
bot            category        label        item                               
rerank_blender likert dialogue emotional    (109,13)_rerank_blender        4   
                               consistent   (109,13)_rerank_blender        4   
                               grammatical  (109,13)_rerank_blender        4   
                               informative  (109,13)_rerank_blender        4   
                               proactive    (109,13)_rerank_blender        4   
...                                                                       ..   
               behavior        follow up    ((438,26)_rerank_blender, 14)  0   
                               topic switch ((438,26)_rerank_blender, 14)  0   
                               ignore       ((438,26)_rerank_blender, 14)  0   
                               irrelevant   ((438,26)_rerank_blender, 14)  0   
                               antisocial   ((438,26)_rerank_blender, 14)  0   

                                                                             1  \
bot            category        label        item                                 
rerank_blender likert dialogue emotional    (109,13)_rerank_blender        4.0   
                               consistent   (109,13)_rerank_blender        4.0   
                               grammatical  (109,13)_rerank_blender        3.0   
                               informative  (109,13)_rerank_blender        4.0   
                               proactive    (109,13)_rerank_blender        3.0   
...                                                                        ...   
               behavior        follow up    ((438,26)_rerank_blender, 14)  NaN   
                               topic switch ((438,26)_rerank_blender, 14)  NaN   
                               ignore       ((438,26)_rerank_blender, 14)  NaN   
                               irrelevant   ((438,26)_rerank_blender, 14)  NaN   
                               antisocial   ((438,26)_rerank_blender, 14)  NaN   

                                                                            2  
bot            category        label        item                               
rerank_blender likert dialogue emotional    (109,13)_rerank_blender       NaN  
                               consistent   (109,13)_rerank_blender       NaN  
                               grammatical  (109,13)_rerank_blender       NaN  
                               informative  (109,13)_rerank_blender       NaN  
                               proactive    (109,13)_rerank_blender       NaN  
...                                                                        ..  
               behavior        follow up    ((438,26)_rerank_blender, 14) NaN  
                               topic switch ((438,26)_rerank_blender, 14) NaN  
                               ignore       ((438,26)_rerank_blender, 14) NaN  
                               irrelevant   ((438,26)_rerank_blender, 14) NaN  
                               antisocial   ((438,26)_rerank_blender, 14) NaN  

[151664 rows x 3 columns]

# 8 Comprehensive Analysis

### Metric Sensitivity

In [78]:
from itertools import combinations
from scipy.stats import ttest_ind

def p_vals(df: pd.DataFrame, test='t', downsample=None):
    """
    :param df: (bot, data point) x 1 -> score
    :param test: statistical test function (t for t test, p for prop test, s for sign test)
    :param downsample: number of samples ber bot to subsample without replacement for the analysis
    :return: p values of test on each bot pair (pd.Series)
    """
    seed = 123
    bots = set(df.index.get_level_values(0))
    num_bots = len(bots)
    bot_pairs = list(combinations(bots, 2))
    result = {}
    for ba, bb in bot_pairs:
        if test == 't':
            if downsample:
                a = df.xs(ba).sample(downsample, random_state=seed).to_numpy().squeeze()
                b = df.xs(bb).sample(downsample, random_state=seed).to_numpy().squeeze()
            else:
                a = df.xs(ba).to_numpy().squeeze()
                b = df.xs(bb).to_numpy().squeeze()
            t, p = ttest_ind(a, b, equal_var=False)
        elif test == 'p':
            if downsample:
                a = df.xs(ba).sample(downsample, random_state=seed).to_numpy().squeeze()
                b = df.xs(bb).sample(downsample, random_state=seed).to_numpy().squeeze()
            else:
                a = df.xs(ba).to_numpy().squeeze()
                b = df.xs(bb).to_numpy().squeeze()
            z, p = proportions_ztest(count=[
                sum(a), sum(b)
            ], nobs=[
                len(a), len(b)
            ])
        elif test == 's':
            # sign test
            comp_data = df.xs((ba, bb), level=[sym.bot, sym.bot_cmp])
            if downsample:
                comp_data = comp_data.sample(downsample, random_state=seed)
            a = comp_data.to_numpy().squeeze() == 1
            b = comp_data.to_numpy().squeeze() == -1
            p = binom_test(sum(a), sum(a)+sum(b), p=0.5)
        else:
            raise ValueError('invalid arg for param "test"')
        result[(ba, bb)] = p
    result_series = pd.Series(result.values(), result)
    return result_series

@to_file
def p_values_comparing_bots(evaluation, downsample=None):
    comp_annotations = get_singly_annotated(evaluation.comparative_annotation_dataframe(), seed=123)
    annotations = get_singly_annotated(evaluation.annotation_dataframe(), seed=123)
    prop_annotations = annotations.xs(
        category.behavior, level=sym.category, drop_level=False
    )
    mean_annotations = annotations.drop(
        index=category.behavior, level=sym.category
    ).drop(
        index=category.comparative, level=sym.category
    )
    mean_ps = mean_annotations.groupby(
        [sym.category, sym.label]
    ).apply(lambda x: p_vals(x, test='t', downsample=downsample))
    prop_ps = prop_annotations.groupby(
        [sym.category, sym.label]
    ).apply(lambda x: p_vals(x, test='p', downsample=downsample))
    comp_groups = comp_annotations.groupby(sym.label)
    comp_ps = comp_groups.apply(lambda x: p_vals(x, test='s', downsample=downsample))
    comp_ps = pd.concat({category.comparative: comp_ps}, names=[sym.category])
    result = pd.concat([prop_ps, mean_ps, comp_ps], axis=0)
    return result

p_values_comparing_bots(data.surge_evaluation, downsample=32, load='results/p_values_comparing_bots_downsampled').round(4)

bart_fid_rag_bcb              \
                                                     emora blender2_3B   
category        label                                                    
behavior        antisocial                             NaN         NaN   
                commonsense contradiction           0.1685      0.3202   
                correct fact                        0.0002      0.0098   
                empathetic                          0.6107      0.7978   
                follow up                           0.0002      0.5986   
                ignore                              0.4911      0.1623   
                incorrect fact                      0.0101      0.0101   
                irrelevant                          0.2002      0.6888   
                lack of empathy                     0.5218      0.2296   
                life info                           0.7404      0.5453   
                partner contradiction               0.6414      0.3017   
                preference info                     0.0016      0.3164   
                redundant                           0.6888      0.3911   
                self contradiction                  0.0101      0.0452   
                topic switch                        0.0079      0.1685   
                uninterpretable                        NaN         NaN   
likert dialogue consistent                          0.0243      0.1380   
                emotional                           0.2968      0.0002   
                engaging                            0.0968      0.0022   
                grammatical                         0.8801      0.1028   
                informative                         0.6013      0.7968   
                proactive                           0.0000      0.0000   
                quality                             0.3231      0.0005   
                relevant                            0.5454      0.0465   
likert turn     consistent                          0.4576      0.0804   
                emotional                           0.0173      0.1112   
                engaging                            0.2696      0.2934   
                grammatical                         0.0210      0.0012   
                informative                         0.0052      0.2691   
                proactive                           0.0013      0.0024   
                quality                             0.6389      0.0107   
                relevant                            0.8537      0.7245   
comparative     consistent                          0.0501      0.5966   
                emotional                           0.2005      0.5847   
                engaging                            0.1102      0.8601   
                grammatical                         0.5716      0.8388   
                informative                         0.2153      0.0201   
                proactive                           0.0107      0.1102   
                quality                             0.4731      0.3771   
                relevant                            0.0501      1.0000   

                                                               emora  \
                                          rerank_blender blender2_3B   
category        label                                                  
behavior        antisocial                           NaN         NaN   
                commonsense contradiction         0.5218      0.6888   
                correct fact                      0.0211      0.1685   
                empathetic                        0.2807      0.8002   
                follow up                         0.0005      0.0010   
                ignore                            0.4911      0.0452   
                incorrect fact                    0.4911         NaN   
                irrelevant                        0.3202      0.0976   
                lack of empathy                   0.2296      0.072

In [79]:
p_values_comparing_bots(data.surge_evaluation, load='results/t_test_p_values_comparing_bots').round(4)

bart_fid_rag_bcb              \
                                                     emora blender2_3B   
category        label                                                    
behavior        antisocial                          0.5597      0.1601   
                commonsense contradiction           0.0000      0.0002   
                correct fact                        0.0000      0.0000   
                empathetic                          0.0000      0.0000   
                follow up                           0.0000      0.5851   
                ignore                              0.3671      0.0436   
                incorrect fact                      0.0000      0.0000   
                irrelevant                          0.0019      0.1866   
                lack of empathy                     0.6253      0.0000   
                life info                           0.0000      0.9618   
                partner contradiction               0.1876      0.9333   
                preference info                     0.0000      0.9816   
                redundant                           0.3665      0.0017   
                self contradiction                  0.0000      0.0000   
                topic switch                        0.0000      0.0002   
                uninterpretable                     0.0001      0.0016   
likert dialogue consistent                          0.0000      0.0138   
                emotional                           0.7603      0.0000   
                engaging                            0.0049      0.0000   
                grammatical                         0.1644      0.0005   
                informative                         0.0209      0.2968   
                proactive                           0.0000      0.0000   
                quality                             0.0003      0.0000   
                relevant                            0.5081      0.0009   
likert turn     consistent                          0.0002      0.0043   
                emotional                           0.2403      0.0000   
                engaging                            0.0000      0.0000   
                grammatical                         0.0000      0.0000   
                informative                         0.0000      0.0000   
                proactive                           0.0000      0.0000   
                quality                             0.0001      0.0000   
                relevant                            0.0693      0.0000   
comparative     consistent                          0.0501      0.5966   
                emotional                           0.2005      0.5847   
                engaging                            0.1102      0.8601   
                grammatical                         0.5716      0.8388   
                informative                         0.2153      0.0201   
                proactive                           0.0107      0.1102   
                quality                             0.4731      0.3771   
                relevant                            0.0501      1.0000   

                                                               emora  \
                                          rerank_blender blender2_3B   
category        label                                                  
behavior        antisocial                        0.0119      0.0587   
                commonsense contradiction         0.0473      0.0008   
                correct fact                      0.0000      0.0000   
                empathetic                        0.0000      0.5285   
                follow up                         0.0000      0.0000   
                ignore                            0.0079      0.0035   
                incorrect fact                    0.0663      0.0023   
                irrelevant                        0.0000      0.0000   
                lack of empathy                   0.0000      0.000

### Predictive Validity

In [80]:
from statsmodels.miscmodels.ordinal_model import OrderedModel
from statsmodels.regression.linear_model import OLS as LinearModel
from statsmodels.discrete.discrete_model import Logit as LogisticModel
from statsmodels.tools.tools import add_constant


def all_dialogue_metrics(data):
    static: pd.DataFrame = data.surge_evaluation.annotation_dataframe()
    static = get_singly_annotated(static, seed=123)
    reindexed = static.reset_index()
    items = reindexed[sym.item]
    dialogues = [e[0] if isinstance(e, tuple) else e for e in items]
    reindexed['dialogue'] = dialogues
    reindexed.set_index(
        [sym.bot, sym.category, sym.label, 'dialogue', sym.item],
        inplace=True, verify_integrity=True
    )
    ld = reindexed.xs(category.likert_dialogue, level=sym.category)
    ld = ld.droplevel(sym.bot).droplevel(sym.item)
    ld.columns = ['score']
    ldq = ld.xs(scale.quality, level=sym.label)
    ldq.columns = ['quality']

    lt = reindexed.xs(category.likert_turn, level=sym.category)
    lt = lt.groupby([sym.label, 'dialogue']).mean()
    lt.columns = ['score']
    ltq = lt.xs(scale.quality, level=sym.label)
    ltq.columns = ['quality']

    be = reindexed.xs(category.behavior, level=sym.category)
    be = be.groupby([sym.label, 'dialogue']).mean()
    be.columns = ['score']

    interactive = data.dialogue_collection.annotation_dataframe()
    idq = interactive.xs((category.likert_dialogue, scale.quality), level=(sym.category, sym.label))
    idq = idq.droplevel(0)

    ds = pd.concat(
        [lt, be, ld],
        keys=[category.likert_turn, category.behavior, category.likert_dialogue],
        names=[sym.category, sym.label, 'dialogue']
    )
    likert_dialogue_quality_features = ds.join(ldq, on='dialogue')
    likert_turn_quality_features = ds.join(ltq, on='dialogue')
    interactive_dialogue_quality_features = ds.join(idq, on='dialogue')
    interactive_dialogue_quality_features.columns = ['score', 'quality']

    interactive_comparisons = data.dialogue_collection.comparative_annotation_dataframe()
    surge_comparisons = get_singly_annotated(data.surge_evaluation.comparative_annotation_dataframe(), seed=123)
    compared_dialogues = surge_comparisons.index.get_level_values('dialogues')
    unique_compared_dialogues = {tuple(x) for x in {frozenset(y) for y in compared_dialogues}}
    comparison_map = dict(unique_compared_dialogues)
    compared_selector = [
        pair in unique_compared_dialogues
        for pair in interactive_comparisons.index.get_level_values('dialogues')
    ]
    comparative: pd.DataFrame = interactive_comparisons.loc[compared_selector, :]
    compared_selector = [
        pair in unique_compared_dialogues
        for pair in surge_comparisons.index.get_level_values('dialogues')
    ]
    surge_comparisons: pd.DataFrame = surge_comparisons.loc[compared_selector, :]
    comparative_quality = comparative.xs(scale.quality, level=sym.label)
    comparative_quality.index = [first for _, _, (first, second) in comparative_quality.index.values]
    comparative_quality.columns = ['quality']
    surge_comparisons.index = pd.MultiIndex.from_arrays(
        list(zip(*[
            (category.comparative, label, left)
            for _, _, label, (left, right) in surge_comparisons.index.values
        ])),
        names=[sym.category, sym.label, 'dialogue']
    )
    surge_comparisons.columns = ['score']
    filtered_ds = ds.loc[[(c, l, d) for c, l, d in ds.index.values if d in comparison_map]]
    compared_features = ds.loc[[(c, l, comparison_map[d]) for c, l, d in filtered_ds.index.values]]
    comparative_features = filtered_ds.to_numpy() - compared_features.to_numpy()
    filtered_ds['diff'] = comparative_features.squeeze().tolist()
    del filtered_ds['score']
    filtered_ds.columns = ['score']
    filtered_ds = pd.concat([filtered_ds, surge_comparisons], axis=0)
    comparative_quality_features = filtered_ds.join(comparative_quality, on='dialogue')
    icq = comparative_quality_features
    icq = icq[icq['quality'] != 0]
    icq.loc[:,'quality'] = (icq['quality'] > 0).astype(int)

    return (
        likert_dialogue_quality_features,
        likert_turn_quality_features,
        icq,
        interactive_dialogue_quality_features
    )

all_dialogue_metrics(data)

def regressions(df, quality_column_name=None, model='linear', adjust_r2=False):
    """
    :param df: dialogue x (*features, quality) -> value
    :return: *(coef, low, high), mcfadden r^2
    """
    if not quality_column_name:
        quality_column_name = df.columns[-1]
    qualities = df[quality_column_name]
    features = [f for f in df.columns if f != quality_column_name]
    if model == 'ordinal':
        model = OrderedModel(qualities, df[features], distr='logit')
        results = model.fit()
        coefs = {f: results.params[f] for f in features}
        prsqrd = results.prsquared
        if adjust_r2:
            prsqrd = 1 - (results.llf - len(features)) / results.llnull
        result = {stat.mcfad_r2: prsqrd, stat.p_of_llr_test: results.llr_pvalue}
    elif model == 'linear':
        x = add_constant(df[features])
        y = qualities
        model = LinearModel(y, x)
        results = model.fit()
        coefs = {f: results.params[f] for f in features}
        rsquared = results.rsquared
        if adjust_r2:
            adjust = lambda r2, f: 1 - (1 - r2) * ((len(y) - 1) / (len(y) - f - 1))
            rsquared = adjust(rsquared, len(features))
        result = {**coefs, stat.r2: rsquared, stat.p_of_f_test: results.f_pvalue}
    elif model == 'logistic':
        x = add_constant(df[features])
        y = qualities.astype(bool)
        model = LogisticModel(y, x)
        results = model.fit()
        coefs = {f: results.params[f] for f in features}
        prsqrd = results.prsquared
        if adjust_r2:
            prsqrd = 1 - (results.llf - len(features)) / results.llnull
        result = {**coefs, stat.mcfad_r2: prsqrd, stat.p_of_llr_test: results.llr_pvalue}
    else:
        raise ValueError('Param "model" must be one of {"linear", "ordinal", "logistic"}')
    return pd.Series(result.values(), result)



/tmp/ipykernel_17595/1045714201.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  icq.loc[:,'quality'] = (icq['quality'] > 0).astype(int)


In [81]:
@to_file
def dialogue_quality_regressions(data):
    ldq, ltq, icq, idq = all_dialogue_metrics(data)
    ldq_groups = ldq.groupby([sym.category, sym.label])
    ltq_groups = ltq.groupby([sym.category, sym.label])
    icq_groups = icq.groupby([sym.category, sym.label])
    idq_groups = idq.groupby([sym.category, sym.label])
    names = ['Predicted', 'Metric']
    linear_compare_result = icq_groups.apply(lambda x: regressions(x, model='logistic'))
    linear_compare_result.columns = pd.MultiIndex.from_arrays(
        [['Interactive Comparison']*3,
        ['LC Coefficient', 'LC Pseudo R-Squared', stat.p_of_llr_test]],
        names=names
    )
    linear_result = ldq_groups.apply(lambda x: regressions(x, model='linear'))
    linear_result.columns = pd.MultiIndex.from_arrays(
        [['Likert Dialogue Quality']*3,
        ['LR Coefficient', 'LR R-Squared', stat.p_of_f_test]],
        names=names
    )
    ordinal_result = ldq_groups.apply(lambda x: regressions(x, model='ordinal'))
    ordinal_result.columns = pd.MultiIndex.from_arrays(
        [['Likert Dialogue Quality']*2,
        ['OR Pseudo R-Squared', stat.p_of_llr_test]],
        names=names
    )
    linear_turn_result = ltq_groups.apply(regressions)
    linear_turn_result.columns = pd.MultiIndex.from_arrays(
        [['Likert Turn Quality']*3,
        ['LR Coefficient', 'LR R-Squared', stat.p_of_f_test]],
        names=names
    )
    interactive_dial_result = idq_groups.apply(regressions)
    interactive_dial_result.columns = pd.MultiIndex.from_arrays(
        [['Interactive Likert']*3,
        ['LR Coefficient', 'LR R-Squared', stat.p_of_f_test]],
        names=names
    )
    result = pd.concat(( linear_compare_result, interactive_dial_result, linear_result, linear_turn_result), axis=1)
    return result.round(5)

regs = dialogue_quality_regressions(
    data,
    load='results/dialogue_quality_regressions'
)
regs

Predicted                                 Interactive Comparison  \
Metric                                            LC Coefficient   
category        label                                              
behavior        antisocial                               0.51392   
                commonsense contradiction               -3.11386   
                correct fact                            -1.17535   
                empathetic                               1.04127   
                follow up                                0.47388   
                ignore                                  -2.11469   
                incorrect fact                          -1.81422   
                irrelevant                              -1.30597   
                lack of empathy                         -3.23250   
                life info                                0.82256   
                partner contradiction                   -2.61231   
                preference info                          0.28662   
                redundant                               -3.01154   
                self contradiction                      -4.16549   
                topic switch                             0.68879   
                uninterpretable                         -6.36913   
comparative     consistent                               0.17569   
                emotional                                0.36613   
                engaging                                 0.34994   
                grammatical                              0.23455   
                informative                             -0.03986   
                proactive                                0.29014   
                quality                                  0.33039   
                relevant                                 0.25995   
likert dialogue consistent                               0.19709   
                emotional                                0.41365   
                engaging                                 0.60412   
                grammatical                              0.09768   
                informative                              0.20144   
                proactive                                0.20918   
                quality                                  0.44739   
                relevant                                 0.30546   
likert turn     consistent                               0.57035   
                emotional                                0.25251   
                engaging                                 0.19076   
                grammatical                              0.30515   
                informative                             -0.42300   
                proactive                                0.70067   
                quality                                  0.56307   
                relevant                                 0.34610   

Predicted                                                      \
Metric                                    LC Pseudo R-Squared   
category        label                                           
behavior        antisocial                            0.00003   
                commonsense contradiction             0.05594   
                correct fact                          0.01423   
                empathetic                            0.01486   
                follow up                             0.00385   
                ignore                                0.01431   
                incorrect fact                        0.02434   
                irrelevant                            0.00915   
                lack of empathy                       0.04713   
                life info                             0.00646   
                partner contradiction                 0.01814   
                preference info                       0.00109   
                redundant                             0.01824   
                self contradiction 

In [82]:
to_plot_regs = regs[[("Likert Dialogue Quality", "LR R-Squared"), ("Likert Dialogue Quality", "P value of F-test")]]
to_plot_regs = to_plot_regs.drop(("likert dialogue", "quality"))
to_plot_regs = to_plot_regs.reset_index()
to_plot_regs

Predicted         category                      label Likert Dialogue Quality  \
Metric                                                           LR R-Squared   
0                 behavior                 antisocial                 0.00013   
1                 behavior  commonsense contradiction                 0.06103   
2                 behavior               correct fact                 0.00010   
3                 behavior                 empathetic                 0.04044   
4                 behavior                  follow up                 0.00440   
5                 behavior                     ignore                 0.03455   
6                 behavior             incorrect fact                 0.01915   
7                 behavior                 irrelevant                 0.04943   
8                 behavior            lack of empathy                 0.04384   
9                 behavior                  life info                 0.00306   
10                behavior      partner contradiction                 0.06043   
11                behavior            preference info                 0.00306   
12                behavior                  redundant                 0.02931   
13                behavior         self contradiction                 0.02745   
14                behavior               topic switch                 0.00001   
15                behavior            uninterpretable                 0.00772   
16             comparative                 consistent                     NaN   
17             comparative                  emotional                     NaN   
18             comparative                   engaging                     NaN   
19             comparative                grammatical                     NaN   
20             comparative                informative                     NaN   
21             comparative                  proactive                     NaN   
22             comparative                    quality                     NaN   
23             comparative                   relevant                     NaN   
24         likert dialogue                 consistent                 0.22558   
25         likert dialogue                  emotional                 0.23022   
26         likert dialogue                   engaging                 0.38103   
27         likert dialogue                grammatical                 0.07243   
28         likert dialogue                informative                 0.17475   
29         likert dialogue                  proactive                 0.27452   
30         likert dialogue                   relevant                 0.28720   
31             likert turn                 consistent                 0.02999   
32             likert turn                  emotional                 0.02943   
33             likert turn                   engaging                 0.02729   
34             likert turn                grammatical                 0.00010   
35             likert turn                informative                 0.00159   
36             likert turn                  proactive                 0.02868   
37             likert turn                    quality                 0.02906   
38             likert turn                   relevant                 0.03776   

Predicted                    
Metric    P value of F-test  
0                   0.82001  
1                   0.00000  
2                   0.84188  
3                   0.00005  
4                   0.18548  
5                   0.00019  
6                   0.00557  
7                   0.00001  
8                   0.00002  
9                   0.26971  
10                  0.00000  
11                  0.26993  
12                  0.00058  
13                  0.00088  
14                  0.94880  
15                  0.07922  
16                      NaN  
17                      NaN  
18                      NaN  
19                      NaN  
20                

In [83]:
regs = prettify(regs, to_csv="results/paper/predictive_validity.csv")

# Incremental Validity

In [84]:
def drop_column_level_duplication(df: pd.DataFrame, columns, levels=None):
    if levels is None:
        levels = list(range(len(columns)))
    level_columns = df.xs(columns, axis=1, level=levels)
    unique = level_columns.iloc[:,0].to_frame()
    unique.columns = [columns]
    dropped = df.drop(columns=columns, level=levels)
    result = pd.concat([dropped, unique], axis=1)
    return result

def multivariate_regression(df: pd.DataFrame, model='linear', adjust_r2=True):
    def apply_regressions(df: pd.DataFrame):
        unstacked = df.unstack([sym.category, sym.label])
        unstacked = drop_column_level_duplication(unstacked, 'quality', 0)
        results = regressions(unstacked, quality_column_name='quality', model=model, adjust_r2=adjust_r2)
        return results
    result = apply_regressions(df)
    result.index = [
        (idx[1] if isinstance(idx, tuple) else idx)
        for idx in result.index.values
    ]
    return result.round(5)

from collections import namedtuple

@to_file
def incremental_regression(
        df: pd.DataFrame,
        categories,
        model='linear',
        beam=1,
        select='backward',
        exclusions=(),
):
    data_points = set(df.index.get_level_values('dialogue'))
    num_data_points = len(data_points)
    Step: type = namedtuple('Step', ('r2', 'p', 'feature'))
    class Path(list):
        def metric(self):
            # if len(self) == 0: return 0
            # else: return self[-1].llr if len(self) == 1 else self[-1].llr / self[-2].llr
            return self.r2
        @property
        def r2(self):
            return self[-1].r2 if self else 0
        # @property
        # def adj_r2(self):
        #     return adjust(self.r2, len(self))
        @property
        def p(self): return self[-1].p if self else 1
        @property
        def features(self): return {x.feature for x in self}
    r2_name = stat.r2 if model=='linear' else stat.mcfad_r2
    p_name = stat.p_of_f_test if model=='linear' else stat.p_of_llr_test
    frontier = [Path()]
    feature_pool = {
        x[:2] for x in df.index.values
        if (not (x in exclusions or x[1] in exclusions))
        and x[0] in categories
    }
    for _ in feature_pool:
        new_frontier = []
        for path in frontier:
            for candidate in feature_pool - path.features:
                if select == 'forward':
                    candidate_features = path.features | {candidate}
                elif select == 'backward':
                    candidate_features = feature_pool - path.features
                else:
                    raise ValueError('param select must be one of {"forward", "backward"}')
                row_mask = [
                    x[:2] in candidate_features
                    and (not (x in exclusions or x[1] in exclusions))
                    and x[0] in categories
                    for x in df.index.values
                ]
                candidate_df = df.loc[row_mask, :]
                candidate_results = multivariate_regression(candidate_df, model=model)
                r2 = candidate_results[r2_name].item()
                p = candidate_results[p_name]
                new_frontier.append(Path([*path, Step(r2, p, candidate)]))
        frontier = sorted(new_frontier, key=lambda x: x.metric(), reverse=True)[:beam]
    result = {
        step.feature: {r2_name: step.r2, p_name: step.p}
        for i, step in enumerate(frontier[0])
    }
    return pd.DataFrame(result.values(), result)


ldq, ltq, icq, idq = all_dialogue_metrics(data)
regs = incremental_regression(
    ldq, (category.likert_turn, category.behavior), beam=1, exclusions=[scale.quality],
    reload='results/dialogue_incremental_regressions'
)
regs

/tmp/ipykernel_17595/1045714201.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  icq.loc[:,'quality'] = (icq['quality'] > 0).astype(int)


R-Squared  P value of F-test
likert turn emotional                    0.12771            0.00000
            engaging                     0.12919            0.00000
behavior    commonsense contradiction    0.12861            0.00000
likert turn consistent                   0.13006            0.00000
behavior    correct fact                 0.13180            0.00000
            irrelevant                   0.13152            0.00000
            redundant                    0.12149            0.00000
            empathetic                   0.11976            0.00000
            life info                    0.10719            0.00000
            preference info              0.10928            0.00000
likert turn grammatical                  0.10947            0.00000
behavior    uninterpretable              0.10510            0.00000
            partner contradiction        0.10443            0.00000
            follow up                    0.08526            0.00000
likert turn informative                  0.08544            0.00000
behavior    incorrect fact               0.08728            0.00000
            antisocial                   0.08284            0.00000
            ignore                       0.08472            0.00000
            topic switch                 0.08132            0.00000
likert turn relevant                     0.08364            0.00000
behavior    lack of empathy              0.06886            0.00000
            self contradiction           0.03795            0.00017
likert turn proactive                    0.02623            0.00067

In [85]:
behavior_regs = incremental_regression(
    ldq, (category.behavior,), beam=10,
    reload='results/behavior_incremental_regressions'
)
behavior_regs

R-Squared  P value of F-test
behavior ignore                       0.12454                0.0
         preference info              0.12628                0.0
         commonsense contradiction    0.12751                0.0
         life info                    0.12746                0.0
         antisocial                   0.12850                0.0
         uninterpretable              0.13069                0.0
         correct fact                 0.13017                0.0
         topic switch                 0.12843                0.0
         follow up                    0.12778                0.0
         redundant                    0.12836                0.0
         lack of empathy              0.12611                0.0
         incorrect fact               0.12450                0.0
         self contradiction           0.11723                0.0
         irrelevant                   0.10375                0.0
         empathetic                   0.09034                0.0
         partner contradiction        0.05807                0.0

In [86]:
behavior_regs = incremental_regression(
    idq, (category.behavior,), beam=10,
    reload='results/behavior_incremental_regressions_interactive'
)
behavior_regs

R-Squared  P value of F-test
behavior preference info              0.22940                0.0
         incorrect fact               0.23104                0.0
         partner contradiction        0.23257                0.0
         life info                    0.23381                0.0
         topic switch                 0.23557                0.0
         antisocial                   0.23743                0.0
         uninterpretable              0.23925                0.0
         follow up                    0.24002                0.0
         correct fact                 0.24065                0.0
         ignore                       0.23886                0.0
         commonsense contradiction    0.23413                0.0
         empathetic                   0.22744                0.0
         redundant                    0.21433                0.0
         irrelevant                   0.18340                0.0
         self contradiction           0.13864                0.0
         lack of empathy              0.10123                0.0

In [87]:
behavior_regs_comp = incremental_regression(
    icq, (category.behavior,), beam=10, model='logistic',
    reload='results/behavior_incremental_regressions_comparative'
)
behavior_regs_comp

Optimization terminated successfully.
         Current function value: 0.583013
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.583013
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.583013
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.583013
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.583013
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.583013
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.583013
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.583013
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.583013
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.583013
  

McFadden's pseudo-R-squared  \
behavior partner contradiction                          0.03212   
         irrelevant                                     0.03964   
         preference info                                0.04716   
         ignore                                         0.05460   
         topic switch                                   0.06153   
         antisocial                                     0.06919   
         incorrect fact                                 0.07676   
         empathetic                                     0.08089   
         correct fact                                   0.08407   
         follow up                                      0.08598   
         redundant                                      0.08801   
         uninterpretable                                0.08891   
         life info                                      0.08918   
         self contradiction                             0.08461   
         lack of empathy                                0.07593   
         commonsense contradiction                      0.04980   

                                    P value of LLR-test  
behavior partner contradiction                  0.00073  
         irrelevant                             0.00044  
         preference info                        0.00025  
         ignore                                 0.00014  
         topic switch                           0.00008  
         antisocial                             0.00004  
         incorrect fact                         0.00002  
         empathetic                             0.00001  
         correct fact                           0.00001  
         follow up                              0.00001  
         redundant                              0.00001  
         uninterpretable                        0.00000  
         life info                              0.00000  
         self contradiction                     0.00000  
         lack of empathy                        0.00001  
         commonsense contradiction              0.00013

In [88]:
likert_turn_regs = incremental_regression(
    ldq, (category.likert_turn,), beam=10, exclusions=[scale.quality],
    reload='results/likert_turn_incremental_regressions'
)
likert_turn_regs

R-Squared  P value of F-test
likert turn grammatical    0.06933            0.00001
            informative    0.07111            0.00000
            engaging       0.07121            0.00000
            emotional      0.06819            0.00000
            consistent     0.06183            0.00000
            proactive      0.05311            0.00001
            relevant       0.03534            0.00009

In [89]:
likert_turn_regs = incremental_regression(
    idq, (category.likert_turn,), beam=10, exclusions=[scale.quality],
    reload='results/likert_turn_incremental_regressions_interactive'
)
likert_turn_regs

R-Squared  P value of F-test
likert turn informative    0.13185                0.0
            grammatical    0.13348                0.0
            emotional      0.13324                0.0
            consistent     0.12935                0.0
            relevant       0.12302                0.0
            proactive      0.09833                0.0
            engaging       0.06453                0.0

In [90]:
likert_turn_regs_comp = incremental_regression(
    icq, (category.likert_turn,), beam=10, model='logistic', exclusions=['quality'],
    reload='results/likert_turn_incremental_regressions_comparative'
)
likert_turn_regs_comp

Optimization terminated successfully.
         Current function value: 0.584976
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.584976
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.584976
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.584976
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.584976
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.584976
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.584976
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.585421
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.585421
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.585421
  

McFadden's pseudo-R-squared  P value of LLR-test
likert turn engaging                         0.09995                  0.0
            emotional                        0.10766                  0.0
            grammatical                      0.11468                  0.0
            relevant                         0.11732                  0.0
            informative                      0.11620                  0.0
            consistent                       0.10270                  0.0
            proactive                        0.08611                  0.0

In [91]:
likert_turn_regs_comp = incremental_regression(
    icq, (category.likert_turn,), beam=10, model='logistic', exclusions=[scale.proactive, scale.quality],
    reload='results/likert_turn_incremental_regressions_comparative_no_proactive_or_quality'
)
likert_turn_regs_comp

Optimization terminated successfully.
         Current function value: 0.624573
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.624573
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.624573
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.624573
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.624573
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.624573
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.624581
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.624581
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.624581
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.624581
  

McFadden's pseudo-R-squared  P value of LLR-test
likert turn emotional                        0.05059              0.00036
            engaging                         0.05843              0.00015
            grammatical                      0.06543              0.00006
            relevant                         0.07104              0.00002
            informative                      0.06306              0.00004
            consistent                       0.04131              0.00040

In [92]:
likert_dialogue_regs = incremental_regression(
    ldq, (category.likert_dialogue,), beam=10, exclusions=['quality'],
    reload='results/likert_dialogue_incremental_regressions'
)
likert_dialogue_regs

R-Squared  P value of F-test
likert dialogue emotional      0.57684                0.0
                informative    0.57189                0.0
                grammatical    0.55887                0.0
                relevant       0.54057                0.0
                proactive      0.51270                0.0
                consistent     0.48414                0.0
                engaging       0.37947                0.0

In [93]:
likert_dialogue_regs = incremental_regression(
    idq, (category.likert_dialogue,), beam=10, exclusions=['quality'],
    reload='results/likert_dialogue_incremental_regressions_interactive'
)
likert_dialogue_regs

R-Squared  P value of F-test
likert dialogue informative    0.10508                0.0
                consistent     0.10736                0.0
                grammatical    0.10936                0.0
                engaging       0.11119                0.0
                proactive      0.11275                0.0
                emotional      0.10154                0.0
                relevant       0.07952                0.0

In [94]:
likert_dialogue_regs_comp = incremental_regression(
    icq, (category.likert_dialogue,), beam=10, model='logistic', exclusions=['quality'],
    reload='results/likert_dialogue_incremental_regressions_comparative'
)
likert_dialogue_regs_comp

Optimization terminated successfully.
         Current function value: 0.601494
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.601494
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.601494
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.601494
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.601494
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.601494
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.601494
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.601515
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.601515
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.601515
  

McFadden's pseudo-R-squared  P value of LLR-test
likert dialogue relevant                         0.07608              0.00002
                grammatical                      0.08390              0.00001
                informative                      0.09159              0.00000
                consistent                       0.09681              0.00000
                proactive                        0.10104              0.00000
                emotional                        0.10452              0.00000
                engaging                         0.10652              0.00000

In [95]:
likert_dialogue_regs_comp = incremental_regression(
    icq, (category.likert_dialogue,), beam=10, model='logistic', exclusions=[scale.engaging, scale.quality],
    reload='results/likert_dialogue_incremental_regressions_comparative_no_engaging_or_quality'
)
likert_dialogue_regs_comp

Optimization terminated successfully.
         Current function value: 0.639971
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.639971
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.639971
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.639971
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.639971
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.639971
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.640620
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.640620
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.640620
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.640620
  

McFadden's pseudo-R-squared  P value of LLR-test
likert dialogue grammatical                      0.02835              0.00381
                relevant                         0.03538              0.00191
                informative                      0.04215              0.00089
                proactive                        0.04828              0.00038
                consistent                       0.05308              0.00016
                emotional                        0.04831              0.00016

In [96]:
comparative_dialogue_regs_comp = incremental_regression(
    icq, (category.comparative,), beam=10, model='logistic', exclusions=[scale.quality],
    reload='results/comparative_incremental_regressions_comparative'
)
comparative_dialogue_regs_comp

Optimization terminated successfully.
         Current function value: 0.663635
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.663635
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.663635
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.663635
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.663635
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.663635
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.663635
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.664634
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.664634
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.664634
  

McFadden's pseudo-R-squared  P value of LLR-test
comparative consistent                      -0.01369              0.16135
            relevant                        -0.00728              0.11867
            proactive                       -0.00027              0.07723
            grammatical                      0.00643              0.04700
            informative                      0.01250              0.02693
            emotional                        0.01662              0.01631
            engaging                         0.01771              0.01072

## Table for Paper

In [97]:
def add_delta(df):
    df['R-Squared'] = (df['Adjusted R-Squared']*100)
    df['delta'] = df['R-Squared'].diff()
    df['delta'] = df['delta'].map('{:.4f}'.format)
    df['R-Squared'] = df['R-Squared'].map('{:.4f}'.format)
    df['R-Squared delta'] = df['R-Squared'] + ' (' + df['delta'] + ')'

final_behavior_regs = behavior_regs.reset_index().rename({'level_1': 'ABC-Eval'}, axis=1).drop(['level_0', 'P value of F-test'], axis=1)
add_delta(final_behavior_regs)

final_likert_turn_regs = likert_turn_regs.reset_index().rename({'level_1': 'Likert Turn'}, axis=1).drop(['level_0', 'P value of F-test'], axis=1)
add_delta(final_likert_turn_regs)

final_likert_dialogue_regs = likert_dialogue_regs.reset_index().rename({'level_1': 'Likert Dialogue'}, axis=1).drop(['level_0', 'P value of F-test'], axis=1)
add_delta(final_likert_dialogue_regs)

final_behavior_regs

KeyError: 'Adjusted R-Squared'

In [ ]:
combined = pd.concat(
    [
        final_behavior_regs[['ABC-Eval', 'R-Squared delta']],
        final_likert_turn_regs[['Likert Turn', 'R-Squared delta']],
        final_likert_dialogue_regs[['Likert Dialogue', 'R-Squared delta']]
    ],
    axis=1)

combined.to_csv('results/paper/incremental_validity.csv', index=False)
combined